## Set up your enviornment

1. Make sure you have the pyodbc driver and do the pip install of the pyodbc package.  Below are helpful links:

- https://learn.microsoft.com/en-us/sql/connect/python/pyodbc/step-1-configure-development-environment-for-pyodbc-python-development?source=recommendations&view=sql-server-ver16

- https://learn.microsoft.com/en-us/sql/connect/odbc/windows/system-requirements-installation-and-driver-files?view=sql-server-ver16#installing-microsoft-odbc-driver-for-sql-server


### Connecting with PyOdbc to a Dedicated SQL Pool Connect
- with sql user & password

In [48]:
import pyodbc
import os

sql_conn = None
server = 'synapsedataload.sql.azuresynapse.net'
database = 'sqlpool'
username = os.getenv('username')
password = os.getenv('password')


try:
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()

    cursor.execute('select * from [dbo].[HumanResources_Department]')
    row = cursor.fetchone() 
    while row: 
        print(row)
        row = cursor.fetchone()

except Exception as e:
    print ("Exception:::", e)



(3, 'Sales', 'Sales and Marketing', datetime.datetime(2008, 4, 30, 0, 0))
(5, 'Purchasing', 'Inventory Management', datetime.datetime(2008, 4, 30, 0, 0))
(10, 'Finance', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0))
(14, 'Facilities and Maintenance', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0))
(2, 'Tool Design', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0))
(12, 'Document Control', 'Quality Assurance', datetime.datetime(2008, 4, 30, 0, 0))
(9, 'Human Resources', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0))
(8, 'Production Control', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0))
(13, 'Quality Assurance', 'Quality Assurance', datetime.datetime(2008, 4, 30, 0, 0))
(6, 'Research and Development', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0))
(7, 'Production', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0))
(11, 'Information Services', 'Ex

### Grant your service princial access to the serverless sql pool

![image info](./screenshot_adding_role.png)

In [61]:
from msal import ConfidentialClientApplication

clientsecret = os.getenv('clientsecret')
tenantid = os.getenv('tenantid')
clientid = os.getenv('clientid')

creds = ConfidentialClientApplication(
    client_id=clientid, 
    authority='https://login.microsoftonline.com/' + tenantid,
    client_credential= clientsecret)

token = creds.acquire_token_for_client(scopes='https://database.windows.net//.default')
print(token)

{'token_type': 'Bearer', 'expires_in': 3599, 'ext_expires_in': 3599, 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2RhdGFiYXNlLndpbmRvd3MubmV0LyIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzE2YjNjMDEzLWQzMDAtNDY4ZC1hYzY0LTdlZGEwODIwYjZkMy8iLCJpYXQiOjE2Nzc2MzEyMzQsIm5iZiI6MTY3NzYzMTIzNCwiZXhwIjoxNjc3NjM1MTM0LCJhaW8iOiJFMlpnWURpeXZkcXdwL1RBck5NbVlTYy9mekIrRGdBPSIsImFwcGlkIjoiMmNiOTQwMDAtM2UxZi00NTdhLTgyZWMtZDdkYTNmOWUwZjg0IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMTZiM2MwMTMtZDMwMC00NjhkLWFjNjQtN2VkYTA4MjBiNmQzLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiNTJiYzU3YWItYzk0MS00YzVjLWI4OWYtZjU0OGVjZjQ3NTczIiwicmgiOiIwLkFVWUFFOEN6RmdEVGpVYXNaSDdhQ0NDMjA5TUhLUUliRF9kSXV0d2JwcXVyYldhOEFBQS4iLCJzdWIiOiI1MmJjNTdhYi1jOTQxLTRjNWMtYjg5Zi1mNTQ4ZWNmNDc1NzMiLCJ0aWQiOiIxNmIzYzAxMy1kMzAwLTQ2OGQtYWM2NC03ZWRhMDgyMGI2ZDMiLCJ1dGkiOiJLZGc3aW9nY3FrLUdtOFJVVHo0TkFBIiwidmVyIjoiMS4wI

In [55]:
import struct

SQL_COPT_SS_ACCESS_TOKEN = 1256 
tokenb = bytes(token["access_token"], "UTF-8")
exptoken = b'';
for i in tokenb:
    exptoken += bytes({i});
    exptoken += bytes(1);
tokenstruct = struct.pack("=i", len(exptoken)) + exptoken;

In [36]:
import pyodbc

connString = 'DRIVER={ODBC Driver 18 for SQL Server};' \
             + 'SERVER=synapsedataload-ondemand.sql.azuresynapse.net;' \
              + 'DATABASE=default;'
conn = pyodbc.connect(connString, attrs_before = { SQL_COPT_SS_ACCESS_TOKEN:tokenstruct});

cursor = conn.cursor()
query="select name from sys.databases"
cursor.execute(query) 
#row = cursor.fetchall()
row = cursor.fetchone() 
while row: 
    print(row)
    row = cursor.fetchone()

('master', )
('default', )


In [56]:
connString = 'DRIVER={ODBC Driver 18 for SQL Server};' \
             + 'SERVER=synapsedataload-ondemand.sql.azuresynapse.net;' \
              + 'DATABASE=default;'
conn = pyodbc.connect(connString, attrs_before = { SQL_COPT_SS_ACCESS_TOKEN:tokenstruct});

cursor = conn.cursor()
query="SELECT TOP (100) * FROM [default].[dbo].[test]"
cursor.execute(query) 
row = cursor.fetchone() 
while row: 
    print(row)
    row = cursor.fetchone()

(680, 'HL Road Frame - Black, 58', 'FR-R92B-58', 'Black', Decimal('1059.3100'), Decimal('1431.5000'), '58', Decimal('1016.04'), 18, 6, datetime.datetime(2002, 6, 1, 0, 0), None, None, b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80\x00\x00\x00\x80\x00\x80\x80\x00\x00\x00\x80\x80\x00\x80\x00\x80\x80\x80\x80\x80\xc0\xc0\xc0\xff\x00\x00\x00\xff\x00\xff\xff\x00\x00\x00\xff\xff\x00\xff\x00\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x003\x00\x00f\x00\x00\x99\x00\x00\xcc\x00\x00\xff\x003\x00\x0033\x003f\x003\x99\x003\xcc\x003\xff\x00f\x00\x00f3\x00ff\x00f\x99\x00f\xcc\x00f\xff\x00\x99\x00\x00\x993\x00\x99f\x00\x99\x99\x00\x99\xcc\x00\x99\xff\x00\xcc\x00\x00\xcc3\x00\xccf\x00\xcc\x99\x00\xcc\xcc\x00\xcc\xff\x00\xff\x00